# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "http://localhost:8888",
        "X-Title": "llm_engineering_course"
    }
)

In [4]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.co

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.

In [8]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [9]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'product page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [11]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 7 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'linkedin page', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'},
  {'type': 'product page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}]}

In [12]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 16 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'careers page', 'url': 'https://huggingface.co/join'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'blog post', 'url': 'https://huggingface.co/blog/ggml-joins-hf'},
  {'type': 'endpoints page', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'u

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [13]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 15 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
GGML and llama.cpp join Hugging Face 🔥
Try HuggingChat Omni – Chat with AI 💬
Get started with Inference in seconds 🚀
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Qwen/Qwen3.5-397B-A17B
Updated
about 7 hours ago
•
303k
•
925
nvidia/personaplex-7b-v1
Updated
8 days ago
•
539k
•
2.17k
Nanbeige/Nanbeige4.1-3B
Updated
2 days ago
•
178k
•
743
zai-org/GLM-5
Updated
10 days ago
•
179k
•
1.46k
MiniMaxAI/MiniMax-M2.5
Updated
8 days ago
•
209k
•
870
Browse 2M+ models
Spaces
Running
on
Zero
MCP
888
Wan2.2 14B Preview
🐌
888
generate a video from an image with a text prompt
Running
on
Ze

In [22]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


"\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGGML and llama.cpp join Hugging Face 🔥\nTry HuggingChat Omni – Chat with AI 💬\nGet started with Inference in seconds 🚀\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3.5-397B-A17B\nUpdated\nabout 7 hours ago\n•\n303k\n•\n925\nnvidia/personaplex-7b-v1\nUpdated\n8 days ago\n•\n539k\n•\n2.17k\nNanbeige/Nanbeige4.1-3B\nUpdated\n2 days ago\n•\n178k\n•\n743\nzai-org/GLM-5\nUpdated\n10 days ago\n•\n179k\n•\n1.46k\nMiniMaxAI/Mini

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 34 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face Brochure

---

## Who We Are

**Hugging Face** is a global AI community dedicated to building the future of machine learning. We provide a collaborative platform that unites researchers, developers, and enterprises to create, share, and evolve machine learning models, datasets, and applications across a wide array of AI modalities—including text, images, video, audio, and 3D.

Our mission is to empower the machine learning community by providing open-source tools, hosting services, and paid compute solutions to accelerate AI innovation.

---

## Our Platform

- **Models:** Explore and collaborate on over 2 million machine learning models, ranging from cutting-edge large language models like Qwen3.5-397B to specialized models for audio and image processing.
- **Datasets:** Access more than 500,000 datasets tailored for various AI tasks such as reasoning, OCR, and spatial mapping.
- **Spaces:** Deploy and interact with over 1 million AI applications built by the community, including demos for AI-generated images, video from text prompts, and custom speech synthesis.
- **Community & Collaboration:** Join a vibrant AI community to share your work, build your ML profile, and collaborate on projects in real-time.
- **Enterprise Services:** Scalable solutions and support tailored for businesses aiming to integrate AI models efficiently.

---

## What We Offer

- **Open Source Stack:** Build faster and smarter leveraging our open-source libraries including Transformers, Diffusers, and Sentence-Transformers.
- **Multi-Modality Support:** Work with text, image, video, audio, and 3D models all on one platform.
- **Easy Inference:** Get started with model inference in seconds using user-friendly tools.
- **Compute Power:** Access paid compute resources optimized for machine learning workloads.

---

## Our Culture

Hugging Face fosters an open, inclusive, and collaborative culture. We believe in building AI **with** the community, not just for it. Our platform is designed to democratize access to AI, encouraging contributors of all experience levels to participate and grow. Innovation thrives through collective knowledge-sharing and transparency.

---

## Customers & Users

Our platform serves a diverse user base including AI researchers, developers, data scientists, educators, and enterprises across various industries. Many use Hugging Face to fast-track their AI research, develop products powered by state-of-the-art models, or engage with the open-source AI ecosystem.

Prominent AI models and applications from the community are hosted on our platform, enabling broad access and adoption worldwide.

---

## Careers

At Hugging Face, we value creativity, collaboration, and a passion for advancing AI technology. We offer opportunities for engineers, researchers, data scientists, community managers, and enterprise solution experts to join our team.

Be part of a mission-driven company shaping the future of AI for everyone. Explore open positions and apply to join a dynamic and inclusive workforce pushing the boundaries of machine learning.

---

## Get Involved

- **Sign Up:** Start your journey by creating a free account on Hugging Face.
- **Explore Models:** Browse millions of pre-trained models available for various AI tasks.
- **Collaborate:** Share your models, datasets, and applications with the global community.
- **Accelerate:** Use paid compute resources to scale your AI projects with ease.

Join Hugging Face—the platform where the machine learning community builds the future.

---

For more information, visit [huggingface.co](https://huggingface.co)

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links


# Hugging Face: Building the Future of AI Together

---

## About Hugging Face

Hugging Face is a vibrant and rapidly growing collaboration platform dedicated to the machine learning (ML) community. It serves as a central hub where ML engineers, scientists, and AI enthusiasts can share, explore, discover, and experiment with open-source ML models, datasets, and applications. The platform supports a vast array of AI modalities including text, image, video, audio, and even 3D, empowering users to move faster in developing AI innovations.

At its core, Hugging Face fosters an open and ethical AI future by bringing together a talented science team and a passionate community that stands at the forefront of the AI revolution. Their platform currently hosts over 2 million models, 500,000 datasets, and 1 million applications, encouraging users to build portfolios and showcase their work globally.

---

## Company Culture

Hugging Face embodies a community-first spirit — collaboration, openness, and innovation are fundamental to their identity. They empower the next generation of AI practitioners to learn, share, and experiment freely, creating an ethical AI ecosystem based on transparency and inclusivity. The company encourages active participation within their extensive community to co-create the future of artificial intelligence.

---

## Products & Services

- **Hugging Face Hub:** The central repository for an extensive collection of open-source ML models, datasets, and applications.
- **Spaces:** Interactive ML apps and demos that let users try AI technologies such as text-to-image, speech generation, video generation, and image editing.
- **Enterprise & Team Solutions:**
  - Tailored AI platform subscriptions starting at $20/user/month.
  - Enterprise-grade security features:
    - Single Sign-On (SSO) integration
    - Granular access control with resource groups
    - Comprehensive audit logs
    - Data residency and management by regions
  - Advanced analytics dashboards for usage monitoring
  - Scalable compute options including ZeroGPU with boosted quotas
  - Private dataset viewers and additional private storage
  - Billing and spending controls for inference providers

---

## Customers & Community

Hugging Face serves a diverse range of users including independent ML practitioners, academic researchers, AI startups, and large enterprises. Its client base benefits from a cutting-edge platform designed for collaboration, scalability, and secure deployment of AI models. The platform is also a thriving community hub where users engage in forums, share resources, and stay up to date with the latest in AI research and applications.

---

## Careers at Hugging Face

Hugging Face offers career opportunities to passionate individuals eager to contribute to the AI revolution. The company values innovation, teamwork, and a commitment to open-source principles. Working at Hugging Face means being part of a pioneer community, collaborating with top scientists and engineers, and making a real impact on the future of technology.

Visit their careers page for current job openings and to explore how you can join this dynamic, mission-driven team.

---

## Connect With Hugging Face

- Website: [huggingface.co](https://huggingface.co)
- GitHub: [github.com/huggingface](https://github.com/huggingface)
- Twitter: [@huggingface](https://twitter.com/huggingface)
- LinkedIn: [Hugging Face](https://linkedin.com/company/huggingface)
- Discord: Join their community chat for real-time discussions

---

## Brand & Visual Identity

- **Colors:** Bright yellow (#FFD21E), orange (#FF9D00), gray (#6B7280)
- **Logos:** Available in SVG, PNG, AI formats for versatile brand use

---

Hugging Face invites you to join the AI community shaping the future — explore their platform, contribute your work, and accelerate your AI journey with the leading open-source AI ecosystem.

In [23]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Hugging Face: The AI Playground Where Everyone’s Invited 🤗

Welcome to **Hugging Face**, the vibrant AI community shaping *the future* — where machine learning engineers, data scientists, artists, and curious minds unite to build, share, and play with AI models, datasets, and applications. Think of it as the ultimate hangout for AI nerds (and proud of it)!

---

### What’s Hugging Face All About?

- **2 Million+ Models and Counting:** From text transformers that write like Shakespeare to image generators that doodle like Picasso, our library of AI models is so vast it could give the Library of Alexandria a run for its money.
- **Datasets Galore:** Need data to train your next AI masterpiece? Browse through 500k+ datasets, including everything from riveting reasoning tests to OCR gems.
- **Spaces: The AI App Store:** Want to generate a video from a simple image? Or maybe create custom speech from text like a futuristic audiobook narrator? Hugging Face Spaces has over a million open apps ready to spark your creativity.
- **Open Source Power:** We champion openness and collaboration. Share your work, experiment with others, and build an ethical, transparent AI future together.
- **Fast & Friendly Compute:** Launch your AI experiments in seconds with our paid compute resources — rocket fuel for your machine learning projects.

---

### Who Calls Hugging Face Home?

- **ML Wizards & Researchers:** From seasoned pros to fresh grads, it’s the go-to hub for the AI community to learn, code, test, and shine.
- **Enterprises:** Serious about AI? Hugging Face Enterprise offers scalable, secure solutions to bring your AI dreams into production.
- **Hobbyists & Innovators:** No PhD? No problem! Tinker with models, join discussions, and be part of a welcoming community where AI is for everyone.

---

### Culture: More Warm Hugs, Less Corporate Bores

- Friendly + collaborative by nature — everyone’s encouraged to speak up, share, and contribute.
- Passion for open science + ethical AI.
- Fast movers who love experimenting—"move fast and build cool stuff" is the unofficial motto.
- A sprinkle of humor and plenty of emojis 🤩 — Because work should feel like play.

---

### Join the Revolution — Careers at Hugging Face

Dream of building AI that changes the world? Hugging Face hires innovators, dreamers, and doers. Whether you’re a software engineer, researcher, community manager, or customer success guru, come code, collaborate, and create in a culture where your ideas matter.

---

### Why Hugging Face?

- Because AI is better *together*.
- Because sharing beats hoarding.
- Because the future is open, creative, diverse, and *hug-filled*.

---

**Ready to take your ML journey to the next level?**

Sign up, browse models, try out Spaces, or just dive into the community buzzing with 100,000+ members—AI magic awaits!

**Website:** https://huggingface.co  
**Colors:** ![#FFD21E](https://via.placeholder.com/15/FFD21E/000000?text=+) `#FFD21E` (Sunshine Yellow), ![#FF9D00](https://via.placeholder.com/15/FF9D00/000000?text=+) `#FF9D00` (Pumpkin Spice), ![#6B7280](https://via.placeholder.com/15/6B7280/FFFFFF?text=+) `#6B7280` (Steel Gray)

---

**Hugging Face — Because AI deserves a hug 🤗**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>